In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import argparse

import logging
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import numpy as np

from torch.autograd import Variable
from modules.gan import Generator, Critic

import copy
import math
import glob
import os
import pickle
import random

import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from func import GPT2Config, OpenAIGPTConfig, XLNetConfig, TransfoXLConfig, BertConfig
from func import GPT2LMHeadModel, GPT2Tokenizer, GPT2ForLatentConnector, GPT2ForLatentConnectorValueHead
from func import OpenAIGPTLMHeadModel, OpenAIGPTTokenizer
from func import XLNetLMHeadModel, XLNetTokenizer
from func import TransfoXLLMHeadModel, TransfoXLTokenizer
from func import BertForLatentConnector, BertTokenizer

from collections import defaultdict
from utils import (TextDataset_Split, TextDataset_2Tokenizers, BucketingDataLoader)
import pdb
from modules.utils import (calc_blue_parallel_func, pad_seq, rollout, rollout_test)
#from transformers.modeling_utils import top_k_top_p_filtering

[nltk_data] Downloading package punkt to /home/harry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from transformers import AutoModel, AutoTokenizer, AutoConfig
import pandas as pd
from matplotlib import pyplot as plt

2022-06-27 14:10:52.928230: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-27 14:10:52.928256: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df_train_l=pd.read_csv("../../yahoo/assigned/train_l.csv", index_col="Unnamed: 0")
df_test_l=pd.read_csv("../../yahoo/assigned/test_l.csv", index_col="Unnamed: 0")
df_u=pd.read_csv("../../yahoo/assigned/u.csv", index_col="Unnamed: 0")
df_train_u=pd.read_csv("../../yahoo/assigned/train_u.csv", index_col="Unnamed: 0")
df_test_u=pd.read_csv("../../yahoo/assigned/test_u.csv", index_col="Unnamed: 0")
df_all = pd.concat([df_train_l, df_test_l, df_u, df_train_u, df_test_u])

In [4]:
train_l =  list(df_train_l.to_records(index=False))
test_l = list(df_test_l.to_records(index=False))
u_list = list(df_u.to_records(index=False))
test_u = list(df_test_u.to_records(index=False))
train_u = list(df_train_u.to_records(index=False))
data_all = list(df_all["0"])

In [5]:
print(train_u[0])

('what is rq?', 'UNK')


In [6]:
label_list = ['UNK',1,2,3,4,5,6,7,8,9,10]

In [7]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples  
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask: 
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples): 
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization 
  for (text, label_mask) in examples:
#     encoded_sent = tokenizer_encoder.convert_tokens_to_ids(tokenizer_encoder.tokenize(text[0]))
#     if len(encoded_sent) < (args.max_seq_length-2):
#         encoded_sent = encoded_sent[0:(args.max_seq_length-2)]
#     encoded_sent = tokenizer_encoder.add_special_tokens_single_sentence(encoded_sent)
#     while len(encoded_sent) < args.max_seq_length:
#         encoded_sent.insert(-1,0)
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=args.max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset), 
              batch_size = args.per_gpu_train_batch_size) # Trains with this batch size.

In [8]:
def generate_data_fake(input_examples):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []

  # Tokenization 
  for text in input_examples:
    encoded_sent = tokenizer.encode(text, add_special_tokens=True, max_length=args.max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)

  # Building the DataLoader
  return input_ids, input_mask_array # Trains with this batch size.

In [9]:
#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

In [10]:
#------------------------------
#   The Generator as in 
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

In [11]:
MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop
ALL_MODELS = sum((tuple(conf.pretrained_config_archive_map.keys()) for conf in (GPT2Config, OpenAIGPTConfig, XLNetConfig, TransfoXLConfig)), ())

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2ForLatentConnectorValueHead, GPT2Tokenizer),
    'bert': (BertConfig, BertForLatentConnector, BertTokenizer)
}

def load_and_cache_examples(args, tokenizer):
    if isinstance(tokenizer, list):
        dataset = TextDataset_2Tokenizers(tokenizer, args, args.train_data_file, block_size=args.block_size)
    else:
        dataset = TextDataset_Split(tokenizer, args, args.train_data_file, block_size=args.block_size)
    return dataset

def build_dataload_and_cache_examples(args, tokenizer):
    if isinstance(tokenizer, list):
        args.batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
        file_path=args.train_data_file
        dataloader = BucketingDataLoader(file_path, args.batch_size, args.max_seq_length, tokenizer, args, bucket=100, shuffle=True)
        print(dataloader)
    else:
        pass 
    return dataloader

def compute_grad_penalty(critic, real_data, fake_data):
    B = real_data.size(0)
    alpha = torch.FloatTensor(np.random.random((B, 1)))
    if args.cuda:
        alpha = alpha.cuda()
    sample = alpha*real_data + (1-alpha)*fake_data
    sample.requires_grad_(True)
    score = critic(sample)

    outputs = torch.FloatTensor(B, 1).fill_(1.0) #args.latent_size
    outputs.requires_grad_(False)
    if args.cuda:
        outputs = outputs.cuda()
    grads = autograd.grad(
        outputs=score,
        inputs=sample,
        grad_outputs=outputs,
        create_graph=True,
        retain_graph=True,
        only_inputs=True)[0]
    grad_penalty = ((grads.norm(2, dim=1) - 1.) ** 2).mean()
    return grad_penalty

def train(epoch, counter):
    
    tr_d_loss = 0
    
    #model_encoder.eval()
    model_decoder.train()
    generator.train()
    #critic.train()
    discriminator.train()
    transformer.train()
    #transformer.eval()
    c_train_loss = 0.
    g_train_loss = 0.
    g_batches = 0
    c_batches = 0
    c_loss_0 = 1
    g_loss_0 = 1
    for i, batch in enumerate(train_loader):
        
        #x = x[0]
        
        args.batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
        
        b_input_ids = batch[0].to(args.device)
        b_input_mask = batch[1].to(args.device)
        b_labels = batch[2].to(args.device)
        b_label_mask = batch[3].to(args.device)
        
        x = b_input_ids
        
        #print("x output is: {}".format(x))
        
        if args.cuda:
            x = x.cuda()
        # Generate noise
        B = args.per_gpu_train_batch_size
#         noise = torch.from_numpy(np.random.normal(0, 1, (B,
#                                  args.latent_size))).float()
        noise = torch.zeros(args.batch_size, noise_size, device=args.device).uniform_(0, 1)
#         if args.cuda:
#             noise = noise.cuda()
            
        # Get original text latent embeddings
#         with torch.no_grad(): 
#             pooled_hidden_fea = model_encoder(x, attention_mask=(x > 0).float())[1]
#             mean, logvar = model_encoder.linear(pooled_hidden_fea).chunk(2, -1)
#             z_real = mean.squeeze(1) 
        model_outputs = transformer(x, attention_mask=(x > 0).float())
        z_real = model_outputs.last_hidden_state[:,0,:]

        # Evaluate and get losses
        z_fake = generator(noise).data
        train_z = rollout_test(model_decoder, z_fake, tokenizer_decoder, args.max_seq_length, args.batch_size, 0, 1)
        x_g, x_g_mask = generate_data_fake(train_z)
        x_g = x_g.to(args.device)
        model_outputs_g = transformer(x_g, attention_mask=(x_g > 0).float())
        z_fake_g = model_outputs_g.last_hidden_state[:,0,:]
        #real_score = critic(z_real)
        #fake_score = critic(z_fake)
        
        D_real_features, D_real_logits, D_real_probs = discriminator(z_real)
        D_fake_features, D_fake_logits, D_fake_probs = discriminator(z_fake_g)
        
        #print("Real logits have these numbers: {}".format(D_real_logits[:,-1]))
        #print("Fake logits have these numbers: {}".format(D_fake_logits[:,-1]))
#         print("real_score critic: {}".format(real_score))
#         print("fake_score critic: {}".format(fake_score))
#         grad_penalty = compute_grad_penalty(critic, z_real.data, z_fake.data)
#         c_loss = -torch.mean(real_score) + torch.mean(fake_score) + \
#                  args.gp_lambda*grad_penalty

        #fake_score = critic(generator(noise))
        #fake_score_d = discriminator(generator(noise))
        #g_loss_old = -torch.mean(fake_score)
        
        # Generator's LOSS estimation
        g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
        g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
        g_loss = g_loss_d + g_feat_reg
        #g_loss = -torch.mean(D_fake_logits[:,-1])
        
        # Disciminator's LOSS estimation
        logits = D_real_logits[:,0:-1]
        log_probs = F.log_softmax(logits, dim=-1)
        
        # The discriminator provides an output for labeled and unlabeled real data
        # so the loss evaluated for unlabeled data is ignored (masked)
        label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
        per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(args.device))
        labeled_example_count = per_example_loss.type(torch.float32).numel()

        # It may be the case that a batch does not contain labeled examples, 
        # so the "supervised loss" in this case is not evaluated
        if labeled_example_count == 0:
          D_L_Supervised = 0
        else:
          D_L_Supervised = torch.div(torch.sum(per_example_loss.to(args.device)), labeled_example_count)
                 
        D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
        D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
        d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U
        
        r_g = abs(((g_loss.item() - g_loss_0) / (g_loss_0 + 0.001))) 
        #r_c = abs(((c_loss.item() - c_loss_0) / (c_loss_0 + 0.001))) 
        
        # Update critic or generator
        #if ((2 + epoch) / epoch) * r_c > r_g:
#             c_optimizer.zero_grad()
#             c_batches += 1
#             c_train_loss += c_loss.item()
#             c_loss.backward(retain_graph=True)
#             c_optimizer.step()
        #if counter % 2 ==0:
        #---------------------------------
        #  OPTIMIZATION
        #---------------------------------
        # Avoid gradient accumulation
        gen_optimizer.zero_grad()
        dis_optimizer.zero_grad()
        # Calculate weigth updates
        # retain_graph=True is required since the underlying graph will be deleted after backward
        g_loss.backward(retain_graph=True)
        d_loss.backward()
        # Apply modifications
        gen_optimizer.step()
        dis_optimizer.step()
        #print("Training discriminator this time")
        #else:
        g_batches += 1
        g_train_loss += g_loss.item()
        
        
        #print("Training generator this time")
        
        counter=counter+1

        # Save the losses to print them later
        tr_d_loss += d_loss.item()

        #c_loss_0 = c_loss.item()
        g_loss_0 = g_loss.item()

        if args.interval > 0 and i % args.interval == 0:
            #logger.info('Epoch: {} | Batch: {}/{} ({:.0f}%) | G Loss: {:.6f} | C Loss: {:.6f}'.format(
            logger.info('Epoch: {} | Batch: {}/{} ({:.0f}%) | G Loss: {:.6f}'.format(
                epoch, args.batch_size*i, len(train_loader.dataset),
                100.*(args.batch_size*i)/len(train_loader.dataset),
                g_loss.item()#, c_loss.item()
            ))
            #test_noise = torch.Tensor(np.random.normal(0, 1, (1, args.latent_size))).to(args.device)
            test_noise = torch.zeros(1, noise_size, device=args.device).uniform_(0, 1)
            test_new_z = generator(test_noise).data
            # create new sent
            test_z = rollout_test(model_decoder, test_new_z, tokenizer_decoder, args.max_seq_length, 1, 0, 1)
            logger.info("Text: {}".format(test_z))
        
    avg_train_loss_d = tr_d_loss / len(train_loader)
    
    print("")
    print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
    
    discriminator.eval()
    transformer.eval()
    model_decoder.eval()
    generatore.eval()
    
    # Tracking variables 
    total_test_accuracy = 0
   
    total_test_loss = 0
    nb_test_steps = 0

    all_preds = []
    all_labels_ids = []

    #loss
    nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)
    
    # Evaluate data for one epoch
    for batch in test_loader:
        
        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(args.device)
        b_input_mask = batch[1].to(args.device)
        b_labels = batch[2].to(args.device)
        
        x = b_input_ids
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
#             pooled_hidden_fea = model_encoder(x, attention_mask=(x > 0).float())[1]
#             mean, logvar = model_encoder.linear(pooled_hidden_fea).chunk(2, -1)
#             z_real = mean.squeeze(1)
            model_outputs = transformer(x, attention_mask=(x > 0).float())
            z_real = model_outputs.last_hidden_state[:,0,:]
            _, logits, probs = discriminator(z_real)
            ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
            filtered_logits = logits[:,0:-1]
            # Accumulate the test loss.
            total_test_loss += nll_loss(filtered_logits, b_labels)
            
        # Accumulate the predictions and the input labels
        _, preds = torch.max(filtered_logits, 1)
        all_preds += preds.detach().cpu()
        all_labels_ids += b_labels.detach().cpu()

    # Report the final accuracy for this validation run.
    all_preds = torch.stack(all_preds).numpy()
    all_labels_ids = torch.stack(all_labels_ids).numpy()
    test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
    print("  Accuracy: {0:.3f}".format(test_accuracy))

    # Calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(test_loader)
    avg_test_loss = avg_test_loss.item()
    
    print("  Test Loss: {0:.3f}".format(avg_test_loss))
    
    c_train_loss /= c_batches + 1
    g_train_loss /= g_batches + 1
    logger.info('* (Train) Epoch: {} | G Loss: {:.4f} | Updates G: {} | Updates C: {}'.format(
        #epoch, g_train_loss, c_train_loss, g_batches, c_batches
        epoch, g_train_loss, g_batches, c_batches
    ))
    return (g_train_loss, test_accuracy, counter)
    #return (g_train_loss, c_train_loss, test_accuracy)

In [12]:
parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--epochs', type=int, default=15)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--gp_lambda', type=int, default=10)
parser.add_argument('--n_layers', type=int, default=20, help="Number of layers of generator and critic")
parser.add_argument('--block_dim', type=int, default=100)
parser.add_argument('--interval', type=int, default=10, help="Steps before logging output")
parser.add_argument('--cuda', type=bool, default=torch.cuda.is_available())

# Optimus parameters
parser.add_argument("--train_data_file", default=None, type=str, required=True,
                    help="The input training data file (a text file).")
parser.add_argument("--valid_data_file", default=None, type=str, required=True,
                    help="The input validation data file (a text file).")
parser.add_argument("--checkpoint_dir", default=None, type=str, required=True,
                    help="The directory where checkpoints are saved.")
parser.add_argument('--generator_dir', default=None, type=str, help="Directory where GAN models are saved")
parser.add_argument("--output_dir", default=None, type=str, required=True,
                    help="The output directory where the model predictions and checkpoints will be written.")
parser.add_argument("--dataset", default='Snli', type=str, help="The dataset.")    
parser.add_argument("--latent_size", default=32, type=int, help="Latent space dimension.")
## Encoder options
parser.add_argument("--encoder_model_type", default="bert", type=str,
                    help="The encoder model architecture to be fine-tuned.")
parser.add_argument("--encoder_model_name_or_path", default="bert-base-cased", type=str,
                    help="The encoder model checkpoint for weights initialization.")
parser.add_argument("--encoder_config_name", default="", type=str,
                    help="Optional pretrained config name or path if not the same as model_name_or_path")
parser.add_argument("--encoder_tokenizer_name", default="", type=str,
                    help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
## Decoder options
parser.add_argument("--decoder_model_type", default="gpt2", type=str,
                    help="The decoder model architecture to be fine-tuned.")
parser.add_argument("--decoder_model_name_or_path", default="bert-base-cased", type=str,
                    help="The decoder model checkpoint for weights initialization.")
parser.add_argument("--decoder_config_name", default="", type=str,
                    help="Optional pretrained config name or path if not the same as model_name_or_path")
parser.add_argument("--decoder_tokenizer_name", default="", type=str,
                    help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
parser.add_argument("--per_gpu_train_batch_size", default=1, type=int,
                    help="Batch size per GPU/CPU for training.")
parser.add_argument("--max_seq_length", default=512, type=int,
                    help="Optional input sequence length before tokenization. The sequence will be dropped if it is longer the max_seq_length")

## Variational auto-encoder(check this)
parser.add_argument("--prompt", type=str, default="")
parser.add_argument("--padding_text", type=str, default="")
parser.add_argument("--length", type=int, default=20)
parser.add_argument("--block_size", default=-1, type=int,
                    help="Optional input sequence length after tokenization."
                         "The training dataset will be truncated in block of this size for training."
                         "Default to the model max input length for single sentence inputs (take into account special tokens).")
parser.add_argument("--do_lower_case", action='store_true',
                    help="Set this flag if you are using an uncased model.")
parser.add_argument("--use_philly", action='store_true',
                    help="Use Philly for computing.")
parser.add_argument('--gloabl_step_eval', type=int, default=661,
                    help="Evaluate the results at the given global step")
# Reinforcement learning parameters
parser.add_argument('--finetune_decoder', type=bool, default=True)
parser.add_argument('--epochs_rl', type=int, default=1000)
parser.add_argument('--batch_size_rl', type=int, default=32)
parser.add_argument('--lr_rl', type=float, default=1e-6)


# Load a trained Encoder model and vocabulary that you have fine-tuned
args = parser.parse_args("--dataset EMNLP \
--checkpoint_dir=checkpoint-508523-768-0 \
--output_dir=checkpoint-508523-768-0 \
--encoder_model_type=bert \
--encoder_model_name_or_path=bert-base-cased \
--decoder_model_type=gpt2 \
--decoder_model_name_or_path=gpt2 \
--train_data_file=../../yahoo/unlabelled/train.txt \
--valid_data_file=../../yahoo/unlabelled/test.txt \
--per_gpu_train_batch_size 6 \
--block_size 100 \
--max_seq_length 24 \
--gloabl_step_eval 508523 \
--latent_size 768 \
--block_dim 100 \
--n_layers 10 \
--interval 50 \
--epochs 200 \
--finetune_decoder True \
--lr_rl 1e-6 \
--epochs_rl 100 \
--batch_size_rl 32".split())

In [13]:
global_step = args.gloabl_step_eval

torch.backends.cudnn.deterministic = True
args.device = torch.device("cuda" if args.cuda else "cpu")
#args.n_gpu = torch.cuda.device_count()
args.n_gpu=1
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)       

args.encoder_model_type = args.encoder_model_type.lower()
args.decoder_model_type = args.decoder_model_type.lower()

output_encoder_dir = os.path.join(args.checkpoint_dir, 'checkpoint-encoder-{}'.format(global_step))
output_decoder_dir = os.path.join(args.checkpoint_dir, 'checkpoint-decoder-{}'.format(global_step)) 
checkpoints = [ [output_encoder_dir, output_decoder_dir] ]

In [14]:
model_name = 'bert-base-cased'
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# Load a trained Encoder model and vocabulary that you have fine-tuned
# encoder_config_class, encoder_model_class, encoder_tokenizer_class = MODEL_CLASSES[args.encoder_model_type]
# model_encoder = encoder_model_class.from_pretrained(output_encoder_dir, latent_size=args.latent_size)
# tokenizer_encoder = encoder_tokenizer_class.from_pretrained(args.encoder_tokenizer_name if args.encoder_tokenizer_name else args.encoder_model_name_or_path, do_lower_case=args.do_lower_case)

#model_encoder.to(args.device)
# if args.block_size <= 0:
#     args.block_size = tokenizer_encoder.max_len_single_sentence  # Our input block size will be the max possible for the model
# args.block_size = min(args.block_size, tokenizer_encoder.max_len_single_sentence)

# Load a trained Decoder model and vocabulary that you have fine-tuned
decoder_config_class, decoder_model_class, decoder_tokenizer_class = MODEL_CLASSES[args.decoder_model_type]
model_decoder = decoder_model_class.from_pretrained(output_decoder_dir, latent_size=args.latent_size)
tokenizer_decoder = decoder_tokenizer_class.from_pretrained(args.decoder_tokenizer_name if args.decoder_tokenizer_name else args.decoder_model_name_or_path, do_lower_case=args.do_lower_case)
model_decoder.to(args.device)
# if args.block_size <= 0:
#     args.block_size = tokenizer_decoder.max_len_single_sentence  # Our input block size will be the max possible for the model
# args.block_size = min(args.block_size, tokenizer_decoder.max_len_single_sentence)

# Chunyuan: Add Padding token to GPT2
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>'}
num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)
logger.info('We have added {} tokens to GPT2'.format(num_added_toks))
model_decoder.resize_token_embeddings(len(tokenizer_decoder))  # Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e. the length of the tokenizer.
assert tokenizer_decoder.pad_token == '<PAD>'

06/27/2022 14:11:08 - INFO - func.configuration_utils -   loading configuration file checkpoint-508523-768-0/checkpoint-decoder-508523/config.json
06/27/2022 14:11:08 - INFO - func.configuration_utils -   Model config {
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "latent_size": 768,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "vocab_size": 50260
}

06/27/2022 14:11:08 - INFO - func.modeling_utils -   loading weights file checkpoint-508523-768-0/checkpoint-decoder-508523/pytorch_model.bin
06/27/2022 14:11:11

In [16]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i

train_examples = train_l
test_examples = test_l

train_label_masks = np.ones(len(train_l), dtype=bool)

if u_list:
  train_examples = train_l + u_list
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(u_list), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])
    
test_u_label_masks = np.zeros(len(test_u), dtype=bool)
train_u_label_masks = np.zeros(len(train_u), dtype=bool)

train_loader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = False)

test_label_masks = np.ones(len(test_examples), dtype=bool)
test_loader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = True, balance_label_examples = False)

/tmp/ipykernel_1269918/4156125318.py:60: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  label_mask_array = torch.tensor(label_mask_array)


In [17]:
out_dropout_rate=0.2
num_hidden_layers_d = 1
hidden_levels_d = [args.latent_size for i in range(0, num_hidden_layers_d)]
discriminator = Discriminator(input_size=args.latent_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
#d_vars = [v for v in discriminator.parameters()]

epsilon = 1e-8
learning_rate_discriminator = 5e-6
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)

discriminator = discriminator.cuda()
transformer = transformer.cuda()

In [18]:
num_hidden_layers_g = 1
hidden_levels_g = [args.latent_size for i in range(0, num_hidden_layers_g)]
noise_size =100

generator = Generator(noise_size=noise_size, output_size=args.latent_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)

g_vars = [v for v in generator.parameters()]
decoder_vars = g_vars + [i for i in model_decoder.parameters()]

learning_rate_generator = 5e-5
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator)

generator = generator.cuda()

In [19]:
#train_loader = build_dataload_and_cache_examples(args, [tokenizer_encoder, tokenizer_decoder]) 
#generator = Generator(args.n_layers, args.block_dim,args.latent_size)
#critic = Critic(args.n_layers, args.block_dim, args.latent_size)

# if args.generator_dir!=None:
#     logger.info("Loading generator and critic")
#     generator.load_state_dict(torch.load(args.generator_dir+'/generator_'+str(args.gloabl_step_eval)+'.th'))
    #critic.load_state_dict(torch.load(args.generator_dir+'/critic_'+str(args.gloabl_step_eval)+'.th'))
    #discriminator.load_state_dict(torch.load(args.generator_dir+'/discriminator_'+str(args.gloabl_step_eval)+'.th'))

#g_optimizer = optim.Adam(generator.parameters(), lr=args.lr, betas=(0.5, 0.999))
#c_optimizer = optim.Adam(critic.parameters(), lr=args.lr, betas=(0.5, 0.999))
#d_optimimizer = optim.Adam(discriminator.parameters(), lr=args.lr, betas=(0.5, 0.999))

# if args.cuda:
#     generator = generator.cuda()
    #critic = critic.cuda()

# logger.info('G Parameters:{}'.format(sum([p.numel() for p in generator.parameters() if \
#                             p.requires_grad])))
# logger.info('C Parameters:{}'.format(sum([p.numel() for p in critic.parameters() if \
#                             p.requires_grad])))
#     logger.info('C Parameters:{}'.format(sum([p.numel() for p in discriminator.parameters() if \
#                                 p.requires_grad])))

best_bleu = 0
reference = list()
with(open(args.valid_data_file,"r")) as valid:
    for sents in valid:
        reference.append(sents.replace("\n", ""))
        
accuracy_array = []
counter = 1

for epoch in range(1, args.epochs + 1):
    #g_loss, c_loss, test_accuracy = train(epoch)
    g_loss, test_accuracy, counter = train(epoch, counter)
    
    accuracy_array.append(test_accuracy)
    
    data_test = list()
    for i in range(2):
        #test_noise = torch.Tensor(np.random.normal(0, 1, (250, args.latent_size))).to(args.device)
        test_noise = torch.zeros(250, noise_size, device=args.device).uniform_(0, 1)
        test_z = generator(test_noise).data
        new_sent = rollout_test(model_decoder, test_z, tokenizer_decoder, args.max_seq_length, 250, 0, 1)
        data_test.extend(new_sent)

    p_reference = random.sample(reference, 500)
    bleu = calc_blue_parallel_func(p_reference, data_test, 2, 500)
    b_bleu = calc_blue_parallel_func(data_test, p_reference, 2, 500)
    logger.info("Bleu-2:{:0.3f} | B-Bleu-2:{:0.3f}".format(bleu, b_bleu))

    print(bleu+b_bleu)
    if (bleu+b_bleu) > best_bleu:
        best_bleu = bleu + b_bleu
        logger.info('* Saving. Best Score:{:0.3f} | Bleu-2:{:0.3f} | B-Bleu-2:{:0.3f}'.format(best_bleu, bleu, b_bleu))
        torch.save(generator.state_dict(), args.output_dir+'/generator_'+str(args.gloabl_step_eval)+'.th')
        #torch.save(critic.state_dict(), args.output_dir+'/critic_'+str(args.gloabl_step_eval)+'.th')
        #torch.save(discriminator.state_dict(), args.output_dir+'/discriminator_'+str(args.gloabl_step_eval)+'.th')

06/27/2022 14:11:17 - INFO - __main__ -   Epoch: 1 | Batch: 0/10200 (0%) | G Loss: 0.118153
06/27/2022 14:11:17 - INFO - __main__ -   Text: ['Soviet Soviet Soviet Recep Recep Liber']
06/27/2022 14:11:32 - INFO - __main__ -   Epoch: 1 | Batch: 300/10200 (3%) | G Loss: 0.435581
06/27/2022 14:11:32 - INFO - __main__ -   Text: ['by Walter']
06/27/2022 14:11:46 - INFO - __main__ -   Epoch: 1 | Batch: 600/10200 (6%) | G Loss: 0.744098
06/27/2022 14:11:46 - INFO - __main__ -   Text: ['dating.']
06/27/2022 14:12:01 - INFO - __main__ -   Epoch: 1 | Batch: 900/10200 (9%) | G Loss: 0.770248
06/27/2022 14:12:01 - INFO - __main__ -   Text: ['to Poverty <BOS> ( <PAD> ,']
06/27/2022 14:12:16 - INFO - __main__ -   Epoch: 1 | Batch: 1200/10200 (12%) | G Loss: 0.699463
06/27/2022 14:12:16 - INFO - __main__ -   Text: ['(2:3) )']
06/27/2022 14:12:31 - INFO - __main__ -   Epoch: 1 | Batch: 1500/10200 (15%) | G Loss: 0.669069
06/27/2022 14:12:31 - INFO - __main__ -   Text: ['and have adapted']
06/27/2022 14


  Average training loss discriminator: 1.092


NameError: name 'generatore' is not defined

In [ ]:
print(max(accuracy_array))

In [ ]:
print(accuracy_array[-1])

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(accuracy_array)
plt.title('HARRYOPTAGAN Yahoo Failed Performance over 200 Training Epochs', fontsize=20)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.xlim(0,200)
plt.show

In [ ]:
df_to_save = pd.DataFrame(accuracy_array)
df_to_save.to_csv('accuracy_array_optagan_yahoo_no_critic_no_encoder_no_agent_transformer.csv')